In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import asyncio
import re
from pathlib import Path

import pandas as pd

from data.samples_to_split import GATSBY_2
from src.builder import AudiobookBuilder
from src.text_split_chain import SplitTextOutput

In [4]:
PANGRAM = "The quick brown fox jumps over the lazy dog"

In [5]:
voices = pd.read_csv("data/11labs_available_tts_voices.csv")
voices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   voice_id                552 non-null    object 
 1   name                    552 non-null    object 
 2   preview_url             552 non-null    object 
 3   owner_id                0 non-null      float64
 4   permission_on_resource  532 non-null    object 
 5   is_legacy               552 non-null    bool   
 6   is_mixed                552 non-null    bool   
 7   accent                  545 non-null    object 
 8   description             20 non-null     object 
 9   age                     550 non-null    object 
 10  gender                  552 non-null    object 
 11  category                513 non-null    object 
 12  language                204 non-null    object 
 13  descriptive             521 non-null    object 
dtypes: bool(2), float64(1), object(11)
memory 

In [6]:
for column in ("age", "accent", "accent", "category"):
    voices[column] = voices[column].str.lower().str.replace(" ", "_").str.replace("-", "_")

In [7]:
voices["is_legacy"].unique()

array([False])

In [8]:
voices["is_mixed"].unique()

array([False])

In [9]:
voices["accent"].unique()

array(['american', 'australian', 'british', 'transatlantic', 'swedish',
       'indian', 'african', 'nigerian', nan, 'canadian', 'new_york',
       'spanish', 'irish', 'german', 'queens_english', 'south_african',
       'malay', 'russian', 'czech', 'us_northeast', 'french', 'tamil'],
      dtype=object)

In [10]:
voices["description"].unique()

array(['expressive', 'confident', 'soft', 'upbeat', 'natural', 'warm',
       'intense', 'articulate', 'seductive', 'friendly', 'casual', 'deep',
       'authoritative', 'trustworthy', nan], dtype=object)

In [11]:
voices["category"].unique()

array(['social_media', 'news', 'conversational', 'narration',
       'characters', 'characters_animation', 'narrative_story',
       'advertisement', 'informative_educational', 'entertainment_tv',
       nan, 'informative_educatio'], dtype=object)

In [12]:
voices["descriptive"].unique()

array([nan, 'upbeat', 'pleasant', 'wise', 'formal', 'sassy', 'confident',
       'modulated', 'calm', 'rough', 'chill', 'deep', 'professional',
       'excited', 'robotic', 'mature', 'soft', 'meditative', 'crisp',
       'casual', 'childish', 'intense', 'grumpy', 'serious', 'hyped',
       'gentle', 'relaxed', 'husky', 'raspy', 'classy', 'neutral',
       'anxious', 'cute', 'cheeky', 'whispery', 'motivational', 'sad'],
      dtype=object)

In [13]:
voices_to_consider = voices.query(
    "gender in ('male', 'female') and category in ('conversational', 'characters') "
    "and accent in ('american', 'australian', 'british', 'canadian', 'new_york', 'transatlantic', 'queens_english')"
).copy()

In [14]:
grouping_columns = ["age", "gender"]
voices_to_consider.groupby(grouping_columns)["voice_id"].count().unstack(fill_value=0)

gender,female,male
age,,
middle_aged,6,11
old,9,6
young,16,9


In [15]:
voices_to_consider[["voice_id", "name", "age", "gender", "category", "accent"]].set_index("voice_id")

,name,age,gender,category,accent
voice_id,,,,,
IKne3meq5aSn9XLyUdCD,Charlie,middle_aged,male,conversational,australian
N2lVS1w4EtoT3dr4eOWO,Callum,middle_aged,male,characters,transatlantic
cgSgspJ2msm6clMCkdW9,Jessica,young,female,conversational,american
cjVigY5qzO86Huf0OWal,Eric,middle_aged,male,conversational,american
iP95p4xoKVk53GoZ742B,Chris,middle_aged,male,conversational,american
1RVpBInY9YUYMLSUQReV,Cody McAvoy,young,male,conversational,american
1btZhL2wthuOhUqvI0bB,Emily - pleasant teen voice,young,female,conversational,british
3lChuqcq4ONNgu6CB9lV,Yasmine,middle_aged,female,conversational,american
8opUN7sGOKbyojnjvNdl,Angela,middle_aged,female,conversational,american


In [30]:
voices_to_consider[["voice_id", "name", "age", "gender", "category", "accent"]].to_excel("voices_to_consider.xlsx", index=False)

In [16]:
builder = AudiobookBuilder()

2024-10-11 06:27:11,096 [INFO] audio-books (select_voice_chain.py): reading voice data from: "data/11labs_available_tts_voices.csv"


In [17]:
text_split_2 = await builder.split_text(GATSBY_2)

2024-10-11 06:27:11,188 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'you are provided with the book sample.\nplease rewrite it and insert xml tags indicating character to whom current phrase belongs.\nfor example: <narrator>I looked at her</narrator><Jill>What are you looking at?</Jill>\n\nNotes:\n- sometimes narrator is one of characters taking part in the action.\nin this case use narrator\'s name (if available) instead of "narrator"\n- if it\'s impossible to identify character name from the text provided, use codes "c1", "c2", etc,\nwhere "c" prefix means character and number is used to enumerate unknown characters\n- all quotes of direct speech must be attributed to characters, for example:\n<Tom>“She’s a nice girl,”</Tom><narrator>said Tom after a moment.</narrator>\nmind that sometimes narrator could also be a character.\n- use ALL available context to determine the character.\nsometimes the character name becomes clear from

In [18]:
print(text_split_2.text_annotated)

<narrator>Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.</narrator>

<narrator>When we came in she held us silent for a moment with a lifted hand.</narrator>

<Jordan>“To be continued,”</Jordan> <narrator>she said, tossing the magazine on the table,</narrator> <Jordan>“in our very next issue.”</Jordan>

<narrator>Her body asserted itself with a restless movement of her knee, and she stood up.</narrator>

<Jordan>“Ten o’clock,”</Jordan> <narrator>she remarked, apparently finding the time on the ceiling.</narrator> <Jordan>“Time for this good girl to go to bed.”</Jordan>

<Daisy>“Jordan’s going to play in the tournament tomorrow,”</Da

In [19]:
def extract_character_phrases(annotated_text: str, character: str) -> SplitTextOutput:
    pattern = re.compile(fr"<{character}>.*</{character}>")
    all_phrases = ""
    for line in annotated_text.splitlines():
        for phrase in pattern.findall(line):
            all_phrases += f"{phrase}\n\n"
    return SplitTextOutput(
        text_raw=all_phrases.replace(f"<{character}>", "").replace(f"</{character}>", ""),
        text_annotated=all_phrases,
    )

In [20]:
male_character = "Tom"
male_phrases = extract_character_phrases(text_split_2.text_annotated, male_character)
print(male_phrases.text_raw)

“She’s a nice girl,” <narrator>said Tom after a moment.</narrator> “They oughtn’t to let her run around the country this way.”

“Her family.”

“Did you give Nick a little heart to heart talk on the veranda?”

“Don’t believe everything you hear, Nick,”




In [21]:
jordan_phrases = extract_character_phrases(text_split_2.text_annotated, "Jordan")
print(jordan_phrases.text_annotated)

<Jordan>“To be continued,”</Jordan> <narrator>she said, tossing the magazine on the table,</narrator> <Jordan>“in our very next issue.”</Jordan>

<Jordan>“Ten o’clock,”</Jordan> <narrator>she remarked, apparently finding the time on the ceiling.</narrator> <Jordan>“Time for this good girl to go to bed.”</Jordan>

<Jordan>“Good night,”</Jordan> <narrator>she said softly.</narrator> <Jordan>“Wake me at eight, won’t you.”</Jordan>

<Jordan>“I will. Good night, Mr. Carraway. See you anon.”</Jordan>

<Jordan>“Good night,”</Jordan> <narrator>called Miss Baker from the stairs.</narrator> <Jordan>“I haven’t heard a word.”</Jordan>




In [22]:
daisy_phrases = extract_character_phrases(text_split_2.text_annotated, "Daisy")
print(daisy_phrases.text_annotated)

<Daisy>“Jordan’s going to play in the tournament tomorrow,”</Daisy> <narrator>explained Daisy,</narrator> <Daisy>“over at Westchester.”</Daisy>

<Daisy>“If you’ll get up.”</Daisy>

<Daisy>“Of course you will,”</Daisy> <narrator>confirmed Daisy.</narrator> <Daisy>“In fact I think I’ll arrange a marriage. Come over often, Nick, and I’ll sort of—oh—fling you together. You know—lock you up accidentally in linen closets and push you out to sea in a boat, and all that sort of thing—”</Daisy>

<Daisy>“Who oughtn’t to?”</Daisy>

<Daisy>“Her family is one aunt about a thousand years old. Besides, Nick’s going to look after her, aren’t you, Nick? She’s going to spend lots of weekends out here this summer. I think the home influence will be very good for her.”</Daisy>

<Daisy>“From Louisville. Our white girlhood was passed together there. Our beautiful white—”</Daisy>

<Daisy>“Did I?”</Daisy> <narrator>She looked at me.</narrator> <Daisy>“I can’t seem to remember, but I think we talked about the 

In [23]:
female_character = "Daisy"
female_phrases = daisy_phrases

In [24]:
async def tts_character_phrases_using_voice(
    audiobook_builder: AudiobookBuilder,
    *,
    split_text: SplitTextOutput,
    character: str,
    voice: str,
) -> Path:
    out_path = Path("data") / "books" / f"{voice}.wav"
    if out_path.exists():
        return out_path
    return await audiobook_builder.audio_generator.generate_audio(
        text_split=split_text,
        character_to_voice={character: voice, "narrator": voice},
        out_path=out_path,
    )

In [25]:
for voice_id in voices_to_consider.query("gender == 'male'")["voice_id"].to_list():
    await tts_character_phrases_using_voice(
        builder,
        split_text=male_phrases,
        character=male_character,
        voice=voice_id,
    )

In [26]:
for voice_id in voices_to_consider.query("gender == 'female'")["voice_id"].to_list():
    await tts_character_phrases_using_voice(
        builder,
        split_text=female_phrases,
        character=female_character,
        voice=voice_id,
    )

2024-10-11 06:27:39,854 [INFO] httpx (_client.py): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-11 06:27:39,856 [INFO] audio-books (generation.py): Successfully modified text with emotional cues: {'modified_text': '<speak><p><s><prosody rate="slow" volume="soft">if you’ll get up.</prosody></s></p></speak>', 'params': {'stability': 0.5, 'similarity_boost': 0.7, 'style': 0.8}}
2024-10-11 06:27:39,969 [INFO] httpx (_client.py): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-11 06:27:39,971 [INFO] audio-books (generation.py): Successfully modified text with emotional cues: {'modified_text': '<speak>\n    <p>\n        <s>\n            <emphasis level="strong">Who oughtn’t to?</emphasis>\n        </s>\n    </p>\n</speak>', 'params': {'stability': 0.5, 'similarity_boost': 0.8, 'style': 0.7}}
2024-10-11 06:27:40,000 [INFO] httpx (_client.py): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1

In [31]:
reviewed_voices = pd.read_excel("reviewed_voices.xlsx")
reviewed_voices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   voice_id  57 non-null     object
 1   name      57 non-null     object
 2   age       57 non-null     object
 3   gender    57 non-null     object
 4   category  57 non-null     object
 5   accent    57 non-null     object
 6   comment   57 non-null     object
dtypes: object(7)
memory usage: 3.2+ KB


In [34]:
ok_voices = reviewed_voices.query("comment in ('ok', 'ok-ish')").copy()
len(ok_voices)

32

In [35]:
ok_voice_ids = ok_voices["voice_id"].unique()

In [36]:
grouping_columns = ["age", "gender"]
voices_to_consider.query("voice_id in @ok_voice_ids").groupby(grouping_columns)["voice_id"].count().unstack(fill_value=0)

gender,female,male
age,,
middle_aged,2,9
old,5,5
young,8,3
